# Minh - Width

Load data

In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm

origin_df = pd.read_csv('data/stage_1.csv', low_memory=False)
origin_df = origin_df.fillna('')
origin_df = origin_df.astype('str')

In [2]:
df = origin_df.copy()

In [3]:
def process_width(df):
    '''
    Input: data frame
    Output: processed data frame
    '''
    metric1 = (df.width != '').sum() / df.shape[0]
    
    def preprocess_width_cell(x):
        if x:
            found = re.search(r'\d+,\d+|\d+\.\d+|\d+', x)         
            if found:
                x = found.group().replace(',', '.')
            else:
                x = ''
        return x
    
    def extract_width(description):
        keywords = ['mặt tiền rộng', 'mặt tiền rộng hơn', 'mặt tiền', 'mt']
        for k in keywords:
            found_index = description.find(k)

            if found_index != -1:
                a = description[found_index + len(k):found_index + len(k) + 10]

                found = re.search(r'\d+,\d+m|\d+,\d+|\d+\.\d+|\d+m|\d+ mét', a)
                if found:
                    found = found.group()

                    return found
        return None
    
    df.width = df.width.apply(preprocess_width_cell)

    new_df = df[['width']].copy()
    
    count_found = 0
    for i, row in tqdm(new_df.iterrows(), total=new_df.shape[0]):
        description = df.at[i, 'description']
        width = row['width']

        if description and width == '':
            description = description.lower()

            found = extract_width(description)
            if found:
                count_found += 1
                new_df.at[i, 'width'] = found

    metric2 = count_found

    df.width = new_df.width.apply(preprocess_width_cell)
    
    metric3 = (df.width != '').sum() / df.shape[0]
    
    print(f"Summary: ")
    print(f"Not null 1: {metric1}")
    print(f"Extracted from description: {metric2}")
    print(f"Not null 2: {metric3}")

    return df

    

In [4]:
df = process_width(df)

  0%|          | 0/314395 [00:00<?, ?it/s]

Summary: 
Not null 1: 0.6700583660681627
Extracted from description: 42029
Not null 2: 0.6015044768523673
